# Lab: Modello ML di base con Dataset Meteo + Integrazione MLflow

Benvenuto in questo laboratorio! Qui imparerai a:

1. **Caricare e preparare un dataset meteo**, con dati di temperatura e umidità.
2. **Addestrare un modello di Machine Learning** con Scikit-learn per prevedere la pioggia.
3. **Valutare il modello** e comprendere i risultati.
4. **Integrare MLflow** per tracciare metriche, parametri e versioni del modello.

Seguiremo un approccio guidato, con spiegazioni dettagliate ad ogni passaggio.  
La prima parte si concentra su Scikit-learn e il dataset meteo. La seconda parte estende il codice esistente con MLflow.

---


## Parte 1: Dai Dati al Modello di Machine Learning (Supervised Learning con Scikit-learn)  

### Obiettivo  
Costruire un **modello di classificazione** che possa prevedere se pioverà, utilizzando come input i dati di **temperatura** e **umidità**. Il modello verrà addestrato con **Scikit-learn**, uno strumento potente per il Machine Learning in Python.

### 1. Preparazione del Dataset  

Prima di addestrare un modello di Machine Learning, è fondamentale pulire i dati, perché valori mancanti o errati possono compromettere le previsioni. Un dataset ben preparato permette al modello di imparare meglio e fornire risultati più accurati.  

Per questo laboratorio utilizzeremo un dataset di esempio:  
[Weather Test Data](https://raw.githubusercontent.com/boradpreet/Weather_dataset/refs/heads/master/Weather%20Test%20Data.csv)  

Il dataset **Weather Test Data** contiene informazioni meteorologiche raccolte in diversi momenti. Ogni riga rappresenta un'osservazione con parametri come **temperatura**, **umidità**, **pressione atmosferica** e altre variabili meteo.  

L'obiettivo di questo dataset è analizzare i pattern climatici e utilizzarli per addestrare un modello di Machine Learning in grado di prevedere condizioni future, come la probabilità di pioggia o variazioni di temperatura.

In [ ]:
import pandas as pd

# URL del dataset 
url = "https://raw.githubusercontent.com/boradpreet/Weather_dataset/refs/heads/master/Weather%20Test%20Data.csv"

# Caricamento del dataset in un dataframe Pandas
df = pd.read_csv(url)

# Mostra le prime righe
df.head(5)



### 2. Esplorazione e Pulizia dei Dati  

Per garantire che il nostro modello funzioni correttamente, dobbiamo prima esaminare e preparare il dataset. Ecco i passaggi fondamentali:  

1. **Controllo dei dati mancanti**: verifichiamo se ci sono valori assenti, poiché potrebbero compromettere l'addestramento del modello. Se necessario, possiamo eliminarli o sostituirli con valori appropriati.  
2. **Conversione della colonna `Label`**: trasformiamo le categorie testuali (*NoRain* e *Rain*) in valori numerici (0 per *NoRain*, 1 per *Rain*), in modo che il modello possa interpretarli correttamente.  
3. **Selezione delle feature principali**: scegliamo solo le colonne più rilevanti (es. temperatura e umidità) per semplificare il modello e migliorare le sue prestazioni.

In [ ]:
# 1. Rimozione dei valori mancanti
df = df.dropna()

# 2. Conversione della colonna 'RainToday' in valori numerici
df['RainToday'] = df['RainToday'].apply(lambda x: 1 if x == 'Yes' else 0)

# 3. Selezione delle feature 
features = ['MinTemp', 'MaxTemp', 'Humidity3pm', 'Humidity9am']

X = df[features]
y = df['RainToday']


### 3. Suddivisione del Dataset in Training e Test  

Per addestrare e valutare correttamente il modello, dividiamo il dataset in due parti:  

- **X (feature)**: contiene le informazioni che useremo per fare previsioni, come **temperatura** e **umidità**.  
- **y (target)**: rappresenta la variabile che vogliamo prevedere, ovvero se pioverà (*Rain*) o meno (*NoRain*).  

Dividiamo i dati in **training set** (80%) e **test set** (20%) per i seguenti motivi:  

1. **Addestramento del modello**  
   - L'80% dei dati viene usato per insegnare al modello a riconoscere i pattern tra le feature e il target.  

2. **Valutazione del modello**  
   - Il restante 20% dei dati non viene usato nell'addestramento, ma serve per testare il modello su dati mai visti prima.  
   - Questo ci permette di capire se il modello è davvero in grado di fare previsioni accurate su nuovi dati.  

3. **Evitare overfitting**  
   - Se testassimo il modello sugli stessi dati con cui è stato addestrato, potremmo ottenere risultati ingannevolmente buoni, perché il modello li avrebbe semplicemente memorizzati.  
   - Usare dati di test separati aiuta a verificare se il modello può generalizzare le sue previsioni a nuovi dati reali.  

Questa suddivisione è un passaggio fondamentale per costruire un modello affidabile e capace di fare previsioni corrette su dati che non ha mai visto prima.

In [ ]:
from sklearn.model_selection import train_test_split

# Suddivisione dei dati (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Dimensioni del dataset di training: {len(X_train)}")
print(f"Dimensioni del dataset di test: {len(X_test)}")

### 4. Creazione e Addestramento del Modello  

Ora che abbiamo preparato i dati, possiamo costruire e addestrare un modello di Machine Learning. Per questo utilizzeremo un classificatore chiamato [**RandomForestClassifier**](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), una delle tecniche più utilizzate per problemi di classificazione.  

#### Perché usiamo **Random Forest**?  
- È un modello basato su **alberi decisionali**, che suddivide i dati in più passaggi per prendere decisioni accurate.  
- È **robusto** e gestisce bene sia dati numerici che categoriali.  
- È meno sensibile ai dati rumorosi rispetto a un singolo albero decisionale, perché combina più alberi per migliorare la precisione.  

Il modello verrà addestrato utilizzando i dati di **temperatura** e **umidità** per prevedere se ci sarà **pioggia** o meno. Dopo l'addestramento, lo testeremo su dati nuovi per valutarne l'accuratezza.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Creazione e addestramento del modello
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

### 5. Valutazione del Modello  

Dopo aver addestrato il modello, dobbiamo verificare quanto è accurato nelle previsioni. Per farlo, calcoliamo **l'accuratezza** e altre metriche di valutazione.  

#### Perché è importante valutare il modello?  
Un modello di Machine Learning non è utile se non sappiamo quanto sia affidabile. La valutazione ci aiuta a capire:  
- **Se il modello sta imparando correttamente dai dati** o se sta solo memorizzando le risposte (overfitting).  
- **Se può essere utilizzato su dati nuovi** e fare previsioni realistiche.  

#### Matrice di Confusione  
Oltre all'accuratezza, utilizzeremo la **matrice di confusione**, un metodo visivo che mostra dove il modello fa previsioni corrette e dove sbaglia.  
- Ci aiuta a individuare **falsi positivi** e **falsi negativi**, che sono errori critici in molti scenari reali.  
- È utile per migliorare il modello, ad esempio regolando soglie di decisione o bilanciando i dati di input.  

Con queste analisi possiamo capire se il nostro modello è pronto per essere utilizzato o se necessita di miglioramenti.

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Predizioni sul set di test
y_pred = model.predict(X_test)

# Calcolo dell'accuratezza ed f1-score
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Accuratezza del modello: {accuracy:.2f}")
print(f"F1-score del modello: {f1:.2f}")

# Report di classificazione
print(classification_report(y_test, y_pred))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Creiamo una heatmap con Seaborn
target_names = ["No Rain", "Rain"]
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=target_names, yticklabels=target_names)

# Aggiungiamo i titoli
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")

# Mostriamo il grafico
plt.show()

### Conclusione (Parte 1)  

In questa prima parte, abbiamo seguito un processo step-by-step per costruire un modello di Machine Learning in grado di prevedere la pioggia. Ecco cosa abbiamo fatto:  

1. **Caricato il dataset meteo** per analizzare temperatura, umidità e altre variabili.  
2. **Pulito e preparato i dati**, gestendo eventuali valori mancanti e trasformando la variabile target in un formato comprensibile al modello.  
3. **Suddiviso il dataset** in dati di training (80%) e di test (20%) per addestrare e valutare il modello correttamente.  
4. **Creato un modello di classificazione** utilizzando **RandomForestClassifier**, un algoritmo potente e robusto.  
5. **Valutato le prestazioni del modello** calcolando l’accuratezza e analizzando la matrice di confusione per identificare eventuali errori.  

Ora che abbiamo costruito il modello base, nella prossima parte esploreremo come integrare **MLflow** per tracciare gli esperimenti e migliorare ulteriormente le prestazioni.

---

## Tech: Installazione e Configurazione di MLflow  

### Obiettivo  
Configurare un'istanza locale di **MLflow** per registrare esperimenti, monitorare le metriche e gestire i modelli di Machine Learning in modo organizzato.  

### 1. Avviare MLflow  

Per avviare MLflow in locale, esegui il seguente comando nel terminale:  

```bash
    mlflow ui
```

Dopo averlo avviato, l'interfaccia grafica sarà accessibile all'indirizzo:  

```
    http://127.0.0.1:5000
```

Questa configurazione permette di **salvare esperimenti e modelli localmente**, consentendo di tracciare le diverse versioni dei modelli, confrontare le metriche di valutazione e ottimizzare i processi di addestramento.  

Nelle prossime sezioni, vedremo come registrare parametri, metriche e modelli direttamente all'interno di MLflow.

### MLflow in Produzione  

In ambienti di produzione e presso i clienti, **MLflow non viene eseguito in locale**, ma viene integrato in un'infrastruttura più solida e scalabile. Questo evita problemi legati alla gestione manuale degli esperimenti e alla persistenza dei dati.  

Le soluzioni più comuni includono:  

- **Docker Compose**  
  - MLflow viene avviato utilizzando un file `docker-compose.yml`, che configura un database backend e uno storage remoto per salvare gli esperimenti.  
  - Questo approccio è utile per ambienti controllati, dove è necessario un setup rapido e replicabile.  
  - Un esempio di implementazione è disponibile nella repository interna:  
    [kiratech/mlops-service-portfolio](https://github.com/kiratech/mlops-service-portfolio/tree/main).  

- **Kubernetes (K8s)**  
  - MLflow viene distribuito su un **cluster Kubernetes**, consentendo una gestione scalabile e centralizzata degli esperimenti.  
  - Questo approccio è ideale per ambienti enterprise, dove sono richiesti elevati livelli di affidabilità, sicurezza e scalabilità.  

Entrambe le soluzioni si basano su un'**architettura multi-container**, che include:  
- **Un database persistente** (es. PostgreSQL o MySQL) per memorizzare i metadati degli esperimenti.  
- **Uno storage S3 o MinIO** per salvare i modelli e gli artefatti, garantendo una gestione sicura e scalabile dei dati.  

Questi approcci assicurano che MLflow possa essere utilizzato in modo affidabile in produzione, integrandosi con infrastrutture cloud o on-premise per una gestione efficace dei modelli di Machine Learning.

---

## Parte 2: Integrazione con MLflow  

Ora estenderemo il codice esistente per **tracciare i nostri esperimenti** utilizzando **MLflow**. Questo ci permetterà di monitorare il processo di addestramento del modello, confrontare le diverse configurazioni e gestire le versioni del modello in modo strutturato.  

### Perché integrare MLflow?  
Con MLflow possiamo:  
- **Registrare i parametri di addestramento** (es. `n_estimators` per Random Forest) per confrontare diverse configurazioni.  
- **Salvare le metriche di valutazione** (es. accuratezza, F1-score) per monitorare le prestazioni del modello.  
- **Archiviare il modello addestrato** per poterlo ricaricare facilmente in futuro e riutilizzarlo senza doverlo riaddestrare.  

### Obiettivo  
Integrare MLflow nel codice esistente per **tracciare e versionare i modelli**, registrando parametri, metriche e artefatti in modo strutturato.  

### 1. Configurazione di MLflow  
Prima di iniziare a tracciare gli esperimenti, impostiamo le variabili necessarie per utilizzare MLflow in questo progetto.

In [ ]:
import mlflow
import mlflow.sklearn

# Impostiamo un nome per l'esperimento
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("weather_classification_experiment")

Verifichiamo che dalla [nostra interfaccia grafica](http://127.0.0.1:5000) il nuovo esperimento sia visibile.

### 2. Registrazione di Parametri, Metriche e Modello  

Con **MLflow**, possiamo salvare e tracciare automaticamente diverse informazioni durante l'addestramento del modello. Questo ci aiuta a confrontare le performance tra diverse configurazioni e a recuperare facilmente i modelli migliori.  

Ecco cosa possiamo registrare:  

- **Parametri** → Valori utilizzati per configurare il modello, come `n_estimators` (numero di alberi in Random Forest) e altri iperparametri.  
- **Metriche** → Indicatori delle prestazioni del modello, come **accuratezza**, **F1-score**, precisione e recall.  
- **Modello** → La versione del modello addestrato, che potrà essere ricaricata e riutilizzata senza bisogno di riaddestramento.  

Registrando questi elementi, possiamo analizzare e confrontare le diverse versioni del modello in modo strutturato e riproducibile.

In [ ]:
# Usando il modello creato nella Parte 1
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from mlflow.models.signature import infer_signature

# Eseguiremo 4 esperimenti per addestrare più modelli 
n_estimators = [1, 10, 100, 500]

for n_e in n_estimators:
    # Creiamo una nuova run MLflow
    with mlflow.start_run():
        # Log Param
        mlflow.log_param("n_estimators", n_e)

        # Creazione del modello
        rf_model = RandomForestClassifier(n_estimators=n_e, random_state=42)
        rf_model.fit(X_train, y_train)

        # Calcolo metriche
        y_pred = rf_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("f1", f1)

        # Creiamo una heatmap con Seaborn
        target_names = ["No Rain", "Rain"]
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(6, 4))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=target_names, yticklabels=target_names)

        # Aggiungiamo i titoli
        plt.xlabel("Predicted Label")
        plt.ylabel("True Label")
        plt.title("Confusion Matrix")

        # Salviamo l'immagine come PNG
        if not os.path.exists("dev/"):
            os.makedirs("dev/")
        plt.savefig("dev/confusion_matrix.png")
        plt.close()
        # Salviamo la confusion matrix su MLFlow come artifatto
        mlflow.log_artifact("dev/confusion_matrix.png")

        # Salviamo il modello
        example_dict = {'MinTemp': 1.1, 'MaxTemp': 1.1, 'Humidity3pm': 1.1, 'Humidity9am': 1.1}
        signature = infer_signature(model_input=example_dict)
        mlflow.sklearn.log_model(rf_model, "random_forest_model", signature=signature)

        print(f"Esperimento concluso. Accuratezza registrata: {accuracy:.2f}")

### 3. Visualizzazione e Confronto dei Risultati  

Dopo aver registrato i parametri, le metriche e i modelli, possiamo utilizzare **MLflow** per esplorare e confrontare i diversi esperimenti.  

MLflow fornisce un’interfaccia web accessibile all’indirizzo:  

```bash
    http://127.0.0.1:5000
```

Accedendo a questa interfaccia, nella sezione **Experiments**, sarà possibile:  
- **Esaminare i parametri** utilizzati in ogni esperimento.  
- **Confrontare le metriche** tra diverse configurazioni di modello.  
- **Visualizzare e scaricare i modelli salvati**, facilitando il riutilizzo e il deployment.  

Questa funzionalità consente di monitorare l’evoluzione delle performance del modello e di identificare rapidamente le configurazioni migliori.

### 4. Caricare un Modello Salvato con MLflow  

MLflow consente di salvare e ricaricare facilmente i modelli addestrati, evitando la necessità di riaddestrarli ogni volta.  

Per recuperare un modello salvato in MLflow, è necessario copiare **l'ID della run** dell'esperimento eseguito. Questo ID identifica in modo univoco ogni esperimento registrato e permette di caricare il modello corrispondente per effettuare previsioni future.  

Questa funzionalità è particolarmente utile per:  
- **Riutilizzare un modello addestrato** senza dover ripetere il training.  
- **Confrontare diverse versioni** di un modello per scegliere quella più performante.  
- **Integrare il modello in applicazioni o API**, senza doverlo ricostruire da zero.  

Nelle prossime sezioni vedremo come eseguire questo processo in pratica.

In [ ]:
import mlflow.sklearn
from sklearn.metrics import accuracy_score

# Inserisci un run_id reale che trovi nell'interfaccia MLflow
RUN_ID = "<run_id_della_tua_run>"

loaded_model = mlflow.sklearn.load_model(f"runs:/{RUN_ID}/random_forest_model")

# Verifichiamo l'accuratezza
y_loaded_pred = loaded_model.predict(X_test)
acc_loaded = accuracy_score(y_test, y_loaded_pred)
print(f"Accuratezza del modello caricato: {acc_loaded:.2f}")

## Conclusioni  

In questo laboratorio, abbiamo seguito un processo completo per costruire e monitorare un modello di Machine Learning applicato ai dati meteorologici. In particolare, abbiamo:  

1. **Creato un modello di classificazione** utilizzando **Scikit-learn**, sfruttando temperatura e umidità per prevedere la pioggia.  
2. **Integrato MLflow** per tracciare i parametri di addestramento, registrare le metriche di valutazione e gestire le versioni del modello in modo strutturato.  
3. **Esplorato e confrontato i risultati** attraverso l’interfaccia di MLflow UI, verificando le diverse configurazioni e caricando un modello salvato per future previsioni.  

Questo approccio ci permette di migliorare il processo di sviluppo dei modelli di Machine Learning, rendendolo più organizzato, riproducibile e scalabile.

## Prossimi Passi  

Ora che abbiamo costruito e tracciato il nostro modello, possiamo esplorare ulteriori miglioramenti e integrare il lavoro in un flusso più avanzato.  

- **Ottimizzazione degli iperparametri**: testare diverse configurazioni di `n_estimators`, `max_depth` e altri parametri del modello, registrando i risultati in **MLflow** per identificare la combinazione migliore.  
- **Automazione con CI/CD**: integrare un sistema di **Continuous Integration/Continuous Deployment (CI/CD)** per addestrare e distribuire automaticamente nuove versioni del modello, riducendo il rischio di errori manuali.  
- **Monitoraggio del modello in produzione**: implementare un sistema di **monitoraggio del drift del modello**, per rilevare eventuali cali di accuratezza nel tempo e decidere quando è necessario riaddestrarlo con nuovi dati.  

Questi passaggi aiutano a trasformare il modello sviluppato in un sistema robusto e affidabile, pronto per essere utilizzato in scenari reali.